In [ ]:
import itertools
import operator

import numpy as np
from astropy import units as u
from magicctapipe.io import load_mc_dl2_data_file
from matplotlib import gridspec
from matplotlib import pyplot as plt
from pyirf.benchmarks import angular_resolution, energy_bias_resolution
from pyirf.cuts import calculate_percentile_cut, evaluate_binned_cut
from pyirf.irf import effective_area_per_energy

In [ ]:
# Configure the pyplot figure
plt.rcParams.update(
    {"figure.figsize": (12, 9), "font.size": 15, "grid.linestyle": "dotted"}
)

# Get the pyplot default color cycle
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

# Load MCs DL2 data

In [ ]:
# ================
# === Settings ===
# ================

input_file_gamma_m = ''

input_file_proton_m= '' 




quality_cuts= f"(disp_diff_mean < {np.sqrt(0.05)})"
irf_type="software"
dl2_weight_type="intensity"
energy_bins=np.logspace(-2,3,15)[2:]





In [ ]:
input_file_gamma = glob.glob(input_file_gamma_m)
input_file_gamma.sort()

print(f"{len(input_file_gamma)} files are found")
input_file_proton = glob.glob(input_file_proton_m)
input_file_proton.sort()


In [ ]:
print("Loading the input files...")

signal_hist=[]
background_hist=[]
signal_hist_6_26=[]
signal_hist_26_46=[]
signal_hist_46_67=[]

for i_file, input_file in enumerate(input_file_gamma):
    # Load the input file
    sig_hist, point_sig, sim_isto_signal =load_mc_dl2_data_file(
        input_file, quality_cuts, irf_type, dl2_weight_type
    )
    
    if point_sig[0]<=26:
        signal_hist_6_26=vstack([signal_hist_6_26,sig_hist])
    elif point_sig[0]<=46:
        signal_hist_26_46=vstack([signal_hist_26_46,sig_hist])
    elif point_sig[0]<=67:
        signal_hist_46_67=vstack([signal_hist_46_67,sig_hist])
    print (len(sig_hist))
    signal_hist=vstack([signal_hist,sig_hist])
    print(type(signal_hist['theta']))
for i_file, input_file in enumerate(input_file_proton):
    # Load the input file
    back_hist, point_back, sim_isto_back =load_mc_dl2_data_file(
        input_file, quality_cuts, irf_type, dl2_weight_type
   
    ) 
    print (len (back_hist))
    background_hist=vstack([background_hist,back_hist])



     

# Show the data table
print(len(background_hist))
print(len(signal_hist))

Gammaness plot

In [ ]:
x=np.array(signal_hist['true_energy'].value)
y=np.array(signal_hist['gammaness'].value)
plt.figure()
plt.xlabel("true TeV")
plt.ylabel("gness TeV")
plt.hist2d(x,y, bins=12,  norm=mpl.colors.LogNorm())
plt.colorbar()

Migration matrix

In [ ]:
x=np.array(signal_hist['reco_energy'].value)
y=np.array(signal_hist['true_energy'].value)
plt.figure()
plt.xscale("log")
plt.yscale("log")
plt.xlabel("reco_E (TeV)")
plt.ylabel("true_E (TeV)")
plt.hist2d(x,y, bins=300,  norm=mpl.colors.LogNorm())
plt.annotate("",
              xy=(0.1,0.1), xycoords='data',
              xytext=(180,180), textcoords='data',
              arrowprops=dict(arrowstyle="-",
                              connectionstyle="arc3,rad=0."), 
              )
plt.colorbar()

# Check the reconstructed energy

In [ ]:
# ================
# === Settings ===
# ================


yscale = "log"

# ============
# === Main ===
# ============

print(f"Energy bins:\n{energy_bins} TeV")

energy_bins_center = (energy_bins[:-1] + energy_bins[1:]) / 2

energy_bins_width = [
    energy_bins[1:] - energy_bins_center,
    energy_bins_center - energy_bins[:-1],
]
print(f"Energy bins center:\n{energy_bins_center} TeV")
plt.figure()
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Number of events")
plt.semilogx()
plt.yscale(yscale)
plt.grid()
E_back=np.array(background_hist["reco_energy"].value)
E_sig=np.array(signal_hist["reco_energy"].value)
# Plot the background energy distribution
plt.hist(
    E_back,
    bins=energy_bins,
    label="proton",
    histtype="step",
    linewidth=3,
)

# Plot the signal energy distribution
plt.hist(
    E_sig,
    bins=energy_bins,
    label="gamma",
    histtype="step",
    linewidth=3,
)

plt.legend()

# Check the gammaness

In [ ]:
# ================
# === Settings ===
# ================

gh_bins = np.linspace(0, 1, 51)
yscale = "log"

# ============
# === Main ===
# ============

plt.figure()
plt.xlabel("Gammaness")
plt.ylabel("Number of events")
plt.yscale(yscale)
plt.grid()
g_back=np.array(background_hist["gammaness"].value)
g_sig=np.array(signal_hist["gammaness"].value)
#Plot the background gammaness distribution
plt.hist(
    g_back,
    bins=gh_bins,
    label="background",
    histtype="step",
    linewidth=2,
)

# Plot the signal gammaness distribution
plt.hist(
    g_sig,
    bins=gh_bins,
    label="signal",
    histtype="step",
    linewidth=2,
)

plt.legend(loc="upper left")


In [ ]:
n_columns = 3
n_rows = int(np.ceil(len(energy_bins[:-1]) / n_columns))

grid = (n_rows, n_columns)
locs = list(itertools.product(range(n_rows), range(n_columns)))

plt.figure(figsize=(20, n_rows * 8))

# Loop over every energy bin
for i_bin, (eng_lolim, eng_uplim) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):

    plt.subplot2grid(grid, locs[i_bin])
    plt.title(f"{eng_lolim:.3f} < energy < {eng_uplim:.3f} [TeV]", fontsize=20)
    plt.xlabel("Gammaness", fontsize=22)
    plt.yscale(yscale)
    plt.grid()

    # Apply the energy cuts
    cond_back_lolim = background_hist["reco_energy"].value > eng_lolim
    cond_back_uplim = background_hist["reco_energy"].value < eng_uplim

    cond_signal_lolim = signal_hist["reco_energy"].value > eng_lolim
    cond_signal_uplim = signal_hist["reco_energy"].value < eng_uplim

    condition_back = np.logical_and(cond_back_lolim, cond_back_uplim)
    condition_signal = np.logical_and(cond_signal_lolim, cond_signal_uplim)

    dt_back = background_hist[condition_back]
    dt_signal = signal_hist[condition_signal]

    # Plot the background gammaness distribution
    if len(dt_back) > 0:
        plt.hist(
             dt_back["gammaness"].value,
             bins=gh_bins,
             label="background",
             histtype="step",
             linewidth=2,
       )

    # Plot the signal gammaness distribution
    if len(dt_signal) > 0:
        plt.hist(
            dt_signal["gammaness"].value,
            bins=gh_bins,
            label="signal",
            histtype="step",
            linewidth=2,
        )

    plt.legend(loc="lower left")


# Apply dynamic gammaness cuts

In [ ]:
# ================
# === Settings ===
# ================

gh_efficiency = 0.9

# ============
# === Main ===
# ============

print(f"Efficiency: {gh_efficiency}")

gh_percentile = 100 * (1 - gh_efficiency)


# Calculate the dynamic gammaness cuts
gh_table_eff = calculate_percentile_cut(
    values=signal_hist["gammaness"],
    bin_values=signal_hist["reco_energy"],
    bins=u.Quantity(energy_bins, u.TeV),
    fill_value=0.0,
    percentile=gh_percentile,
)

gh_cuts_eff = gh_table_eff["cut"].value
print(f"\n efficiency gammaness cuts:\n{gh_cuts_eff}")

In [ ]:
plt.figure()
plt.title(f"cuts(g/h efficiency = {gh_efficiency})")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Gammaness cut")
plt.semilogx()
plt.grid()


# Plot the dynamic gammaness cuts
plt.errorbar(
    x=energy_bins_center,
    y=gh_cuts_eff,
    xerr=energy_bins_width,
    label="gamma efficiency",
    marker="o",
)



In [ ]:


print("\nApplying the gammaness cuts...")




mask_gh_eff = evaluate_binned_cut(
    values=signal_hist["gammaness"],
    bin_values=signal_hist["reco_energy"],
    cut_table=gh_table_eff,
    op=operator.ge,
)

data_eff_gcut = signal_hist[mask_gh_eff]
print(f"--> Number of events: {len(data_eff_gcut)}")


mask_gh_eff_26 = evaluate_binned_cut(
    values=signal_hist_6_26["gammaness"],
    bin_values=signal_hist_6_26["reco_energy"],
    cut_table=gh_table_eff,
    op=operator.ge,
)

data_eff_gcut_26 = signal_hist_6_26[mask_gh_eff_26]
  
    
mask_gh_eff_46 = evaluate_binned_cut(
    values=signal_hist_26_46["gammaness"],
    bin_values=signal_hist_26_46["reco_energy"],
    cut_table=gh_table_eff,
    op=operator.ge,
)

data_eff_gcut_46 = signal_hist_26_46[mask_gh_eff_46]



mask_gh_eff_67 = evaluate_binned_cut(
    values=signal_hist_46_67["gammaness"],
    bin_values=signal_hist_46_67["reco_energy"],
    cut_table=gh_table_eff,
    op=operator.ge,
)

data_eff_gcut_67 = signal_hist_46_67[mask_gh_eff_67]



# Check the reconstructed direction

In [ ]:
# ================
# === Settings ===
# ================

theta2_bins = np.linspace(0, 0.5, 91)
yscale = "log"

# ============
# === Main ===
# ============

plt.figure()
plt.title(f"g/h efficiency = {gh_efficiency}")
plt.xlabel("Theta2 [deg$^2$]")
plt.ylabel("Number of events")
plt.yscale(yscale)
plt.grid()



eff_gcut=np.array(data_eff_gcut["theta"].value)

# Plot the signal theta2 distribution
plt.hist(
    np.square(eff_gcut),
    bins=theta2_bins,
    label="signal;efficiency",
    histtype="step",
    linewidth=2,
)

plt.legend()


In [ ]:
n_columns = 3
n_rows = int(np.ceil(len(energy_bins[:-1]) / n_columns))

grid = (n_rows, n_columns)
locs = list(itertools.product(range(n_rows), range(n_columns)))

plt.figure(figsize=(20, n_rows * 8))

# Loop over every energy bin
for i_bin, (eng_lolim, eng_uplim) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):

    plt.subplot2grid(grid, locs[i_bin])
    plt.title(f"{eng_lolim:.3f} < energy < {eng_uplim:.3f} [TeV]")
    plt.xlabel("Theta2 [deg$^2$]")
    plt.yscale(yscale)
    plt.grid()

    # Apply the energy cuts
    cond_eff_lolim = data_eff_gcut["reco_energy"].value > eng_lolim
    cond_eff_uplim = data_eff_gcut["reco_energy"].value < eng_uplim

    
    
    condition_eff = np.logical_and(cond_eff_lolim, cond_eff_uplim)

    
    dt_eff = data_eff_gcut[condition_eff]
    dte=np.array(dt_eff["theta"].value)
    
    # Plot the theta2 distribution
    if len(dt_eff) > 0:
        plt.hist(
            np.square(dte),
            bins=theta2_bins,
            label="signal;eff",
            histtype="step",
            linewidth=2,
        )

    plt.legend(loc="upper right")


# Check the angular resolution

In [ ]:


# Calculate the angular resolution
angres_table_eff = angular_resolution(
    data_eff_gcut, u.Quantity(energy_bins, u.TeV), energy_type="reco"
)

angres_eff = angres_table_eff["angular_resolution"].value
print(f"\n angular resolution:\n{angres_eff} deg")



In [ ]:
plt.figure()
gs = gridspec.GridSpec(4, 1)


plt.set_title(f"angular resolution(g/h efficiency = {gh_efficiency})")
plt.set_ylabel("Angular resolution (68% cont.) [deg]")
plt.semilogx()
plt.grid()




# Plot the angular resolution
plt.errorbar(
    x=energy_bins_center,
    y=angres_eff,
    xerr=energy_bins_width,
    label="signal;eff",
    marker="o",
)


plt.legend()


# Apply dynamic theta cuts

In [ ]:
# ================
# === Settings ===
# ================
theta_efficiency_68=0.68
theta_efficiency = 0.8

# ============
# === Main ===
# ============



theta_percentile = 100 * theta_efficiency
theta_percentile_68 = 100 * theta_efficiency_68


# Calculate the dynamic theta cuts
theta_table_eff = calculate_percentile_cut(
    values=data_eff_gcut["theta"],
    bin_values=data_eff_gcut["reco_energy"],
    bins=u.Quantity(energy_bins, u.TeV),
    fill_value=data_eff_gcut["theta"].unmasked.max(),
    percentile=theta_percentile,
)
theta_table_eff_68= calculate_percentile_cut(
    values=data_eff_gcut["theta"],
    bin_values=data_eff_gcut["reco_energy"],
    bins=u.Quantity(energy_bins, u.TeV),
    fill_value=data_eff_gcut["theta"].unmasked.max(),
    percentile=theta_percentile_68,
)

theta_cuts_eff = theta_table_eff["cut"]
theta_cuts_eff_68 = theta_table_eff_68["cut"]

theta_cut_eff = theta_table_eff["cut"].value
theta_cut_eff_68 = theta_table_eff_68["cut"].value
print(f"\n(0.8 efficiency) theta cuts:\n{theta_cuts_eff}")
print(f"\n(0.68 efficiency) theta cuts:\n{theta_cuts_eff_68}")

In [ ]:
plt.figure()
plt.title(f"theta cuts (g/h eff. = {gh_efficiency})")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Theta cut [deg]")
plt.semilogx()
plt.grid()

# Plot the 68% dynamic theta cuts
plt.errorbar(
    x=energy_bins_center,
    y=theta_cuts_eff_68,
    xerr=energy_bins_width,
    label="68%",
    marker="o",
)

# Plot the 80% dynamic theta cuts
plt.errorbar(
    x=energy_bins_center,
    y=theta_cuts_eff,
    xerr=energy_bins_width,
    label="80%",
    marker="o",
)

plt.legend(loc="upper right")


In [ ]:



# Apply dynamic theta cuts
print("\nApplying the theta cuts to signal;eff...")

mask_theta_eff = evaluate_binned_cut(
    values=data_eff_gcut["theta"],
    bin_values=data_eff_gcut["reco_energy"],
    cut_table=theta_table_eff,
    op=operator.le,
)

data_eff_gtcuts = data_eff_gcut[mask_theta_eff]
print(f"--> Number of events: {len(data_eff_gtcuts)}")

# Check the effective area

Warning: effective area depends on azimuth: MCs with azimuth in a narrow range (~30 deg) must be provided as imput or you will get meaningless results

In [ ]:


# Calculate the low zenith effective area
aeff_eff_26 = effective_area_per_energy(
    selected_events=data_eff_gcut_26,
    simulation_info=sim_isto_signal,
    true_energy_bins=u.Quantity(energy_bins, u.TeV),
)

print(f"\neffective area:\n{aeff_eff_26}")

In [ ]:

# Calculate the mid zenith effective area
aeff_eff_46 = effective_area_per_energy(
    selected_events=data_eff_gcut_46,
    simulation_info=sim_isto_signal,
    true_energy_bins=u.Quantity(energy_bins, u.TeV),
)

print(f"\neffective area:\n{aeff_eff_46}")

In [ ]:


# Calculate the high zenith effective area
aeff_eff_67 = effective_area_per_energy(
    selected_events=data_eff_gcut_67,
    simulation_info=sim_isto_signal,
    true_energy_bins=u.Quantity(energy_bins, u.TeV),
)

print(f"\neffective area:\n{aeff_eff_67}")

In [ ]:
plt.figure()
plt.title(f"g/h eff. = {gh_efficiency}, theta eff. = {theta_efficiency}")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Effective area [m$^2$]")
plt.loglog()
plt.grid()


# Plot the effective area
plt.errorbar(
    x=energy_bins_center,
    y=aeff_eff_26.value,
    xerr=energy_bins_width,
    label="zenith < 26 deg",
    marker="o",
)


# Plot the effective area
plt.errorbar(
    x=energy_bins_center,
    y=aeff_eff_46.value,
    xerr=energy_bins_width,
    label="26 deg < zenith < 46 deg",
    marker="o",
)



# Plot the effective area
plt.errorbar(
    x=energy_bins_center,
    y=aeff_eff_67.value,
    xerr=energy_bins_width,
    label="46 deg < zenith < 67 deg",
    marker="o",
)

plt.legend(loc="lower right")


# Check the energy bias and resolution

In [ ]:



# Calculate the  signal energy bias and resolution
engres_table_eff = energy_bias_resolution(
    data_eff_gcut, u.Quantity(energy_bins, u.TeV), energy_type="reco"
)

engbias_eff = engres_table_eff["bias"].value
engres_eff = engres_table_eff["resolution"].value

print(
    f"\n signal energy bias:\n{engbias_eff}"
    f"\n\n signal energy resolution:\n{engres_eff}"
)



In [ ]:
plt.figure()
gs = gridspec.GridSpec(4, 1)


plt.set_title(f"E bias and resolution(g/h eff. = {gh_efficiency}, theta eff. = {theta_efficiency})")
plt.set_ylabel("Energy bias and resolution")
plt.semilogx()
plt.grid()





# Plot the signal energy bias and resolution
plt.errorbar(
    x=energy_bins_center,
    y=engres_eff,
    xerr=energy_bins_width,
    label="Energy resolution",
    marker="o",
    color=colors[1],
)

plt.errorbar(
    x=energy_bins_center,
    y=engbias_eff,
    xerr=energy_bins_width,
    label="Energy bias",
    marker="o",
    linestyle="--",
    color=colors[1],
)


plt.legend()


# Plots: cut effects on background

In [ ]:

back_gn_cut_eff=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    
    mask_table_eff=(background_hist["reco_energy"].value >np.repeat(eng_lo,(len(background_hist)))) \
        & (background_hist["reco_energy"].value <np.repeat(eng_hi,(len(background_hist)))) \
            & (background_hist["gammaness"].value>np.repeat(gh_table_eff['cut'][i_bin],(len(background_hist))))
    back_masked_gn_eff=background_hist[mask_table_eff]
    back_gn_cut_eff=vstack([back_masked_gn_eff,back_gn_cut_eff])
    print(len(back_masked_gn_eff))
print("tot",len(back_gn_cut_eff))
back_gn_cut_eff

In [ ]:

ttheta_table_eff=Table(theta_table_eff)
ttheta_table_eff_68=Table(theta_table_eff_68)

In [ ]:
#theta 68%,eff
back_theta_cut_eff_68=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    
    mask_table_theta_eff_68=(back_gn_cut_eff["reco_energy"].value >np.repeat(eng_lo,(len(back_gn_cut_eff)))) \
        & (back_gn_cut_eff["reco_energy"].value <np.repeat(eng_hi,(len(back_gn_cut_eff)))) \
            & (back_gn_cut_eff["theta"].value < np.repeat(theta_cut_eff_68[i_bin],(len(back_gn_cut_eff))))
    back_masked_theta_eff_68=back_gn_cut_eff[mask_table_theta_eff_68]
    back_theta_cut_eff_68=vstack([back_masked_theta_eff_68,back_theta_cut_eff_68])
    print(len(back_masked_theta_eff_68))
print("tot",len(back_theta_cut_eff_68))

In [ ]:
#theta 80%,eff
back_theta_cut_eff_80=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    
    mask_table_theta_eff_80=(back_gn_cut_eff["reco_energy"].value >np.repeat(eng_lo,(len(back_gn_cut_eff)))) \
        & (back_gn_cut_eff["reco_energy"].value <np.repeat(eng_hi,(len(back_gn_cut_eff)))) \
            & (back_gn_cut_eff["theta"].value < np.repeat(theta_cut_eff[i_bin],(len(back_gn_cut_eff))))
    back_masked_theta_eff_80=back_gn_cut_eff[mask_table_theta_eff_80]
    back_theta_cut_eff_80=vstack([back_masked_theta_eff_80,back_theta_cut_eff_80])
    print(len(back_masked_theta_eff_80))
print("tot",len(back_theta_cut_eff_80))

Plot

In [ ]:
plt.figure()
plt.title("background")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Number of events")
plt.semilogx()
plt.yscale(yscale)
plt.grid()

theta_80_eff=np.array(back_theta_cut_eff_80["reco_energy"].value)
theta_68_eff=np.array(back_theta_cut_eff_68["reco_energy"].value)
gn_eff=np.array(back_gn_cut_eff["reco_energy"].value)
tot_back=np.array(background_hist["reco_energy"].value)
# before cuts
plt.hist(
    tot_back,
    bins=energy_bins,
    label="total",
    histtype="step",
    linewidth=3,
    color="b"
)




#gammaness cut
plt.hist(
    gn_eff,
    bins=energy_bins,
    label="eff_cut",
    histtype="step",
    linewidth=3,
    color="y"
)


#theta 80%, eff
plt.hist(
    theta_80_eff,
    bins=energy_bins,
    label="eff_cut; theta_80",
    histtype="step",
    linewidth=3,
    color="c"
)
#theta 68%, eff
plt.hist(
    theta_68_eff,
    bins=energy_bins,
    label="eff_cut; theta_68",
    histtype="step",
    linewidth=3,
    color="k"
)


plt.legend()



# Plots: cut effects on signal

In [ ]:


sig_gn_cut_eff=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    mask_E=np.logical_and((signal_hist["reco_energy"].value >np.repeat(eng_lo,(len(signal_hist)))), \
        (signal_hist["reco_energy"].value <np.repeat(eng_hi,(len(signal_hist)))))
    mask_table_eff_sig=np.logical_and(mask_E,(signal_hist["gammaness"].value>np.repeat(gh_table_eff['cut'][i_bin],(len(signal_hist)))))
    sig_masked_gn_eff=signal_hist[mask_table_eff_sig]
    sig_gn_cut_eff=vstack([sig_masked_gn_eff,sig_gn_cut_eff])
    print(len(sig_masked_gn_eff))
print("tot",len(sig_gn_cut_eff))
sig_gn_cut_eff

In [ ]:

ttheta_table_eff=Table(theta_table_eff)
ttheta_table_eff_68=Table(theta_table_eff_68)

In [ ]:
#theta 68%,eff
sig_theta_cut_eff_68=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    mask_table_theta_eff_68_sig=(sig_gn_cut_eff["reco_energy"].value >np.repeat(eng_lo,(len(sig_gn_cut_eff)))) \
        & (sig_gn_cut_eff["reco_energy"].value <np.repeat(eng_hi,(len(sig_gn_cut_eff)))) \
            & (sig_gn_cut_eff["theta"].value<np.repeat(theta_cut_eff_68[i_bin],(len(sig_gn_cut_eff))))
    sig_masked_theta_eff_68=sig_gn_cut_eff[mask_table_theta_eff_68_sig]
    sig_theta_cut_eff_68=vstack([sig_masked_theta_eff_68,sig_theta_cut_eff_68])
    print(len(sig_masked_theta_eff_68))
print("tot",len(sig_theta_cut_eff_68))

In [ ]:
#theta 80%,eff
sig_theta_cut_eff_80=QTable()
for i_bin, (eng_lo, eng_hi) in enumerate(zip(energy_bins[:-1], energy_bins[1:])):
    
    mask_table_theta_eff_80_sig=(sig_gn_cut_eff["reco_energy"].value >np.repeat(eng_lo,(len(sig_gn_cut_eff)))) \
        & (sig_gn_cut_eff["reco_energy"].value <np.repeat(eng_hi,(len(sig_gn_cut_eff)))) \
            & (sig_gn_cut_eff["theta"].value<np.repeat(theta_cut_eff[i_bin],(len(sig_gn_cut_eff))))
    sig_masked_theta_eff_80=sig_gn_cut_eff[mask_table_theta_eff_80_sig]
    sig_theta_cut_eff_80=vstack([sig_masked_theta_eff_80,sig_theta_cut_eff_80])
    print(len(sig_masked_theta_eff_80))
print("tot",len(sig_theta_cut_eff_80))

Plot

In [ ]:
plt.figure()
plt.title("signal")
plt.xlabel("Reconstructed energy [TeV]")
plt.ylabel("Number of events")
plt.semilogx()
plt.yscale(yscale)
plt.grid()

theta_80_eff_sig=np.array(sig_theta_cut_eff_80["reco_energy"].value)
theta_68_eff_sig=np.array(sig_theta_cut_eff_68["reco_energy"].value)
gn_eff_sig=np.array(sig_gn_cut_eff["reco_energy"].value)
sE=np.array(signal_hist["reco_energy"].value)
# before cuts
plt.hist(
    sE,
    bins=energy_bins,
    label="total",
    histtype="step",
    linewidth=3,
    color="b"
)




#gammaness cut
plt.hist(
    gn_eff_sig,
    bins=energy_bins,
    label="eff_cut",
    histtype="step",
    linewidth=3,
    color="y"
)

#theta 80%, eff
plt.hist(
    theta_80_eff_sig,
    bins=energy_bins,
    label="eff_cut; theta_80",
    histtype="step",
    linewidth=3,
    color="c"
)
#theta 68%, eff
plt.hist(
    theta_68_eff_sig,
    bins=energy_bins,
    label="efff_cut; theta_68",
    histtype="step",
    linewidth=3,
    color="k"
)
plt.legend()
